In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow.compat.v1 as tf
import glob
import keras
!pip install music21
import music21 as ms
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


     |████████████████████████████████| 18.5 MB 3.5 MB/s 
  Created wheel for music21: filename=music21-5.7.2-py3-none-any.whl size=22024602 sha256=ad22a67fe83a8b1fd339723bdd5afd63cc049877b711beb23de149a99d43b2c9
  Stored in directory: /root/.cache/pip/wheels/d8/a1/8c/52ddcf152401fa8f947f995cd2e9fa10c1a40de639218a5fe2
Successfully built music21
/kaggle/input/classical-piano-dataset/Fiend_Battle_(Piano).mid
/kaggle/input/classical-piano-dataset/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
/kaggle/input/classical-piano-dataset/tifap.mid
/kaggle/input/classical-piano-dataset/costadsol.mid
/kaggle/input/classical-piano-dataset/redwings.mid
/kaggle/input/classical-piano-dataset/ViviinAlexandria.mid
/kaggle/input/classical-piano-dataset/ff7themep.mid
/kaggle/input/classical-piano-dataset/Ff4-BattleLust.mid
/kaggle/input/classical-piano-dataset/roseofmay-piano.mid
/kaggle/input/classical-piano-dataset/ff1battp.mid
/kaggle/input/classical-piano-dataset/AT.mid
/kaggle/input/classical-pi

In [2]:
notes = []
c = 1
for file in glob.glob('../input/classical-piano-dataset/*.mid'):
    print(c)
    c = c + 1
    song = ms.converter.parse(file)
    parse_notes = song.flat.notes
    for item in parse_notes:
        if isinstance(item, ms.note.Note):
            notes.append(str(item.pitch))
        elif isinstance(item, ms.chord.Chord):
            notes.append('.'.join(str(n) for n in item.normalOrder))
len(notes)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


60498

In [3]:
sequence_length = 150
pitches = sorted(set(item for item in notes))
pitch_dict = dict((note,number) for number, note in enumerate(pitches))

LSTM_input = []
LSTM_output = []

for i in range(0, len(notes) - sequence_length):
    inp_sequence = notes[i:i+sequence_length]
    output_char = notes[i+sequence_length]
    
    LSTM_input.append([pitch_dict[item] for item in inp_sequence])
    LSTM_output.append(pitch_dict[output_char])

LSTM_input = np.reshape(LSTM_input, (len(LSTM_input), sequence_length, 1))
LSTM_output = keras.utils.np_utils.to_categorical(LSTM_output)

n_vocab = np.unique(np.array(notes))

In [4]:
LSTM_model = keras.models.Sequential()
layer1 = keras.layers.LSTM(256, input_shape = [LSTM_input.shape[1], LSTM_input.shape[2]], return_sequences = True)
layer2 = keras.layers.Dropout(0.1)
layer3 = keras.layers.LSTM(512, return_sequences = True)
layer4 = keras.layers.Dropout(0.1)
layer5 = keras.layers.LSTM(256)
layer6 = keras.layers.Dense(256)
layer7 = keras.layers.Dropout(0.1)
layer8 = keras.layers.Dense(len(n_vocab))
layer9 = keras.layers.Softmax()

LSTM_model.add(layer1)
LSTM_model.add(layer2)
LSTM_model.add(layer3)
LSTM_model.add(layer4)
LSTM_model.add(layer5)
LSTM_model.add(layer6)
LSTM_model.add(layer7)
LSTM_model.add(layer8)
LSTM_model.add(layer9)

LSTM_model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam())

In [5]:
os.chdir('/kaggle/working')
filename = '/kaggle/working/weights-{epoch:02d}-{loss:.4f}.hdf5'
check = keras.callbacks.ModelCheckpoint(filename, monitor = 'loss', verbose = 0, save_best_only = True, mode = 'min')
callback_list = [check]

LSTM_model.fit(LSTM_input, LSTM_output, epochs = 45, batch_size = 256, callbacks = callback_list)

Epoch 1/45
60348/60348 [==============================] - 98s 2ms/step - loss: 4.5981
Epoch 2/45


/opt/conda/lib/python3.6/site-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


60348/60348 [==============================] - 95s 2ms/step - loss: 4.3219
Epoch 3/45
60348/60348 [==============================] - 96s 2ms/step - loss: 4.1094
Epoch 4/45
60348/60348 [==============================] - 96s 2ms/step - loss: 3.8905
Epoch 5/45
60348/60348 [==============================] - 95s 2ms/step - loss: 3.6575
Epoch 6/45
60348/60348 [==============================] - 96s 2ms/step - loss: 3.4355
Epoch 7/45
60348/60348 [==============================] - 95s 2ms/step - loss: 3.2331
Epoch 8/45
60348/60348 [==============================] - 96s 2ms/step - loss: 3.0474
Epoch 9/45
60348/60348 [==============================] - 96s 2ms/step - loss: 2.8706
Epoch 10/45
60348/60348 [==============================] - 96s 2ms/step - loss: 2.7108
Epoch 11/45
60348/60348 [==============================] - 95s 2ms/step - loss: 2.5575
Epoch 12/45
60348/60348 [==============================] - 96s 2ms/step - loss: 2.4196
Epoch 13/45
60348/60348 [==============================] - 97s

In [6]:
start = np.random.randint(0, len(LSTM_input) -1)

int_to_note = dict((number,note) for number, note in enumerate(pitch_dict))

starting_tune = LSTM_input[start]
pred = []

for i in range(200):
    pred_input = np.reshape(starting_tune, (1, len(starting_tune), 1))
    pred_output = LSTM_model.predict(pred_input, verbose = 0)
    
    number = np.argmax(pred_output)
    note = int_to_note[number]
    pred.append(note)
    #print(len(starting_tune))
    starting_tune = np.append(starting_tune,number)
    #print(len(starting_tune))
    starting_tune = starting_tune[1:len(starting_tune)]
    #print(len(starting_tune))

In [7]:
offset = 0
output_notes = []

for note_pred in pred:
    if ('.' in note_pred) or note_pred.isdigit():
        chord_note = note_pred.split('.')
        notes = []
        for current in chord_note:
            new_note = ms.note.Note(int(current))
            new_note.storedInstrument = ms.instrument.Piano()
            notes.append(new_note)
        new_chord = ms.chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = ms.note.Note(note_pred)
        new_note.storedInstrument = ms.instrument.Piano()
        new_note.offset = offset
        output_notes.append(new_note)
    offset = offset + 0.5

In [8]:
midi_stream = ms.stream.Stream(output_notes)
midi_stream.write('midi', fp = 'the_final_output.mid')

'the_final_output.mid'